# 🔎 Vector Search

Vector search is a technique used in information retrieval where data is represented as vectors in a high-dimensional space. These vectors are typically dense representations of the data, capturing both semantic and syntactic features.

In this example we will build a small vector database that enables natural language lookup of terms. A vector search is not the same as an LLM query; it's faster and more focused, but cannot capture nuance or generate content. Both techniques have tradeoffs but can be used together to build powerful information systems.

## 🛠️ Setup

We will start by installing the packages we need and creating a connection to Ollama's embeddings model.
> *Note:* This is not an LLM, but rather a much smaller model for returning vectors based on input strings.

In [ ]:
#r "nuget: System.Linq.Async"
#r "nuget: Microsoft.SemanticKernel.Connectors.Ollama, 1.32.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.InMemory, 1.33.0-preview"
#r "nuget: Microsoft.Extensions.AI.Ollama, 9.0.1-preview.1.24570.5"
#r "nuget: OllamaSharp"

using Microsoft.Extensions.VectorData;
using Microsoft.SemanticKernel.Connectors.InMemory;
using Microsoft.SemanticKernel.Embeddings;
using OllamaSharp;

using System.Net.Sockets;

var ollamaApiUrl = "http://localhost:11434";
try {
  var tcp = new TcpClient();
  tcp.SendTimeout = 100;
  tcp.Connect("host.docker.internal", 11434);
  ollamaApiUrl = "host.docker.internal";
} catch { }

var embeddingsModel = "nomic-embed-text";

// Create an embedding generation service.
var client = new OllamaApiClient(ollamaApiUrl, embeddingsModel);
var textEmbeddingGenerationService = client.AsEmbeddingGenerationService();

// Construct an InMemory vector store.
var vectorStore = new InMemoryVectorStore();

Installed Packages Microsoft.Extensions.AI.Ollama, 9.0.1-preview.1.24570.5 Microsoft.SemanticKernel.Connectors.InMemory, 1.33.0-preview Microsoft.SemanticKernel.Connectors.Ollama, 1.32.0-alpha OllamaSharp, 5.0.2 System.Linq.Async, 6.0.1

## 📒 Define our glossary

We need to create an annotated object to let the vector database know how our objects look and which items need to be indexed for filtering.

In [ ]:
private sealed class Glossary
{
    [VectorStoreRecordKey]
    public ulong Key { get; set; }

    [VectorStoreRecordData(IsFilterable = true)]
    public string Category { get; set; } = string.Empty;

    [VectorStoreRecordData]
    public string Term { get; set; } = string.Empty;

    [VectorStoreRecordData]
    public string Definition { get; set; } = string.Empty;

    [VectorStoreRecordVector(1536)]
    public ReadOnlyMemory<float> DefinitionEmbedding { get; set; }
}

## 🚛 Create glossary definitions

Now we will create the list of glossary entries. While this is a hardcoded list, it could just as easily be data retrieved from a conventional database, scraped from websites, gathered from files, or really any other source.

In [ ]:
private static IEnumerable<Glossary> CreateGlossaryEntries()
{
    yield return new Glossary
    {
        Key = 1,
        Category = "External Definitions",
        Term = "API",
        Definition = "Application Programming Interface. A set of rules and specifications that allow software components to communicate and exchange data."
    };

    yield return new Glossary
    {
        Key = 2,
        Category = "Core Definitions",
        Term = "Connectors",
        Definition = "Connectors allow you to integrate with various services provide AI capabilities, including LLM, AudioToText, TextToAudio, Embedding generation, etc."
    };

    yield return new Glossary
    {
        Key = 3,
        Category = "External Definitions",
        Term = "RAG",
        Definition = "Retrieval Augmented Generation - a term that refers to the process of retrieving additional data to provide as context to an LLM to use when generating a response (completion) to a user’s question (prompt)."
    };
}

## 💾 Load the data

Now we will call out to the model service (in our case Ollama) to have it generate encodings. Semantic Kernel handles most of the "heavy lifting" in this case.

In [ ]:
// Get and create collection if it doesn't exist.
var collection = vectorStore.GetCollection<ulong, Glossary>("skglossary");
await collection.CreateCollectionIfNotExistsAsync();

// Create glossary entries and generate embeddings for them.
var glossaryEntries = CreateGlossaryEntries().ToList();
var tasks = glossaryEntries.Select(entry => Task.Run(async () =>
{
    entry.DefinitionEmbedding = await textEmbeddingGenerationService.GenerateEmbeddingAsync(entry.Definition);
}));
await Task.WhenAll(tasks);

// Upsert the glossary entries into the collection and return their keys.
var upsertedKeysTasks = glossaryEntries.Select(x => collection.UpsertAsync(x));
await Task.WhenAll(upsertedKeysTasks);

## 🔎 Search the data

Finally we have made it through all the steps to create a vector database.
1. We defined our type
2. Created some data
3. Loaded that data into the vector data store

Now let's see the power of a vector store in action by running a query against it!

In [ ]:
// Search the collection using a vector search.
var searchString = "What is an api?";
var searchVector = await textEmbeddingGenerationService.GenerateEmbeddingAsync(searchString);
var searchResult = await collection.VectorizedSearchAsync(searchVector, new() { Top = 1 });
var resultRecords = await searchResult.Results.ToListAsync();

Console.WriteLine(@$"
Search string: {searchString}
Result:
          Id: {resultRecords.First().Record.Key}
        Term: {resultRecords.First().Record.Term}
  Definition: {resultRecords.First().Record.Definition}");



Search string: What is an api?
Result:
          Id: 1
        Term: API
  Definition: Application Programming Interface. A set of rules and specifications that allow software components to communicate and exchange data.


Wow, that was a lot faster than querying a large language model! 🏎️
We got back exactly the glossary record we wanted. Now you can start to see how classifying information and feeding it into a vector store can be helpful. We can use vector query results both pre and post LLM queries to fine tune either the initial query or the results.

## 📚 Read More

[Optimizing Retrieval for RAG Apps: Vector Search and Hybrid Techniques](https://techcommunity.microsoft.com/blog/educatordeveloperblog/optimizing-retrieval-for-rag-apps-vector-search-and-hybrid-techniques/4138030)